# Combine All ATP Synthase and Proton Pump Models
---

In [1]:
from biocrnpyler import *
#from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, Multimer
import numpy as np
import pylab as plt 
%matplotlib inline

import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
bokeh.io.output_notebook()

import colorcet

from subsbml import createNewSubsystem, createSubsystem

Loading BokehJS ...

In [2]:
palette = colorcet.b_glasbey_category10
palette.remove( '#d62628')

## Combine ATP synthase tx, tl, membrane integration, atp synthesis models
---

In [3]:
# atp synthase models
#ss1_atpsyn_txtl = createSubsystem('atpsynthase_tx_tl.xml', subsystemName = 'atpsyn_txtl')
ss1_atpsyn_txtl = createSubsystem('atpsynthase_simple_txtl.xml', subsystemName = 'atpsyn_txtl')


ss2_atpsyn_mi = createSubsystem('atpsynthase_membrane_integration.xml', subsystemName = 'atpsyn_mi')
ss3_atpsynthesis = createSubsystem('atp_synthesis_through_atpsynthase.xml', subsystemName = 'atpsynthesis')

# proton pump ss
# ss4_pp_txtl = createSubsystem('proton_pump_tx_tl.xml', subsystemName = 'pp_txtl')
# ss5_pp_mi = createSubsystem('proton_pump_membrane_integration.xml', subsystemName = 'pp_mi')
# ss6_protontransport = createSubsystem('proton_transport_through_proton_pump.xml', subsystemName = 'protontransport')


In [4]:
#atp_subsystem = ss1_atpsyn_txtl + ss2_atpsyn_mi + ss3_atpsynthesis
atp_subsystem = ss1_atpsyn_txtl + ss2_atpsyn_mi + ss3_atpsynthesis


In [5]:
#atp_subsystem.getSpeciesByName('protein_atpsyn_b')

In [6]:
# spec_list_10 = ['metabolite_atp', 'proton_outside_H', 'dna_atpsyn', 'protein_RNAP_machinery',
#                'protein_Ribo_machinery', 'protein_RNAase_machinery']
spec_list_10 = ['dna_atpsyn', 'proton_outside_H', 'metabolite_atp']
for i,spec in enumerate(spec_list_10):
    atp_subsystem.setSpeciesAmount(spec,10)

In [7]:
timepoints = np.linspace(0,10,100)
results, _ = atp_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

In [8]:
results.columns

Index(['rna_atpsyn_atpsyn_txtlbiocrnpyler_429022',
       'dna_atpsyn_atpsyn_txtlbiocrnpyler_429022',
       'protein_atpsyn_atpsyn_txtlbiocrnpyler_429022_1_combined',
       'proton_outside_H_atpsynthesisbiocrnpyler_634411',
       'proton_inside_H_atpsynthesisbiocrnpyler_634411',
       'protein_atpsyn_b_atpsyn_mibiocrnpyler_494589_1_combined',
       'metabolite_adp_atpsynthesisbiocrnpyler_634411',
       'metabolite_atp_atpsynthesisbiocrnpyler_634411',
       'metabolite_pi_atpsynthesisbiocrnpyler_634411', 'time'],
      dtype='object')

In [9]:
ATP_id = atp_subsystem.getSpeciesByName('metabolite_atp').getId()
ADP_id = atp_subsystem.getSpeciesByName('metabolite_adp').getId()

outside_H_id = atp_subsystem.getSpeciesByName('proton_outside_H').getId()
inside_H_id = atp_subsystem.getSpeciesByName('proton_inside_H').getId()
atp_synthase_b_protein_id = atp_subsystem.getSpeciesByName('protein_atpsyn_b').getId()
atp_synthase_f_protein_id = atp_subsystem.getSpeciesByName('protein_atpsyn').getId()

rna_atpsyn_id = atp_subsystem.getSpeciesByName('rna_atpsyn').getId()

dna_atpsyn_id = atp_subsystem.getSpeciesByName('dna_atpsyn').getId()

In [10]:
results.columns

Index(['rna_atpsyn_atpsyn_txtlbiocrnpyler_429022',
       'dna_atpsyn_atpsyn_txtlbiocrnpyler_429022',
       'protein_atpsyn_atpsyn_txtlbiocrnpyler_429022_1_combined',
       'proton_outside_H_atpsynthesisbiocrnpyler_634411',
       'proton_inside_H_atpsynthesisbiocrnpyler_634411',
       'protein_atpsyn_b_atpsyn_mibiocrnpyler_494589_1_combined',
       'metabolite_adp_atpsynthesisbiocrnpyler_634411',
       'metabolite_atp_atpsynthesisbiocrnpyler_634411',
       'metabolite_pi_atpsynthesisbiocrnpyler_634411', 'time'],
      dtype='object')

In [11]:
p = bokeh.plotting.figure(width = 450, height = 300)
plot_list = [ATP_id, outside_H_id, inside_H_id, atp_synthase_b_protein_id]
plot_str = ['atp', 'outside_H', 'inside_H', 'atp synthase bound']
# plot_list = [atp_synthase_b_protein_id,rna_atpsyn_id, atp_synthase_f_protein_id,dna_atpsyn_id,  ]
# plot_str = [ 'bound atp synthase', 'atpsyn rna','free atp synthase', 'atpsyn dna', ]
for i,item in enumerate(plot_list):
    p.line(timepoints, results[item], line_width = 2, legend_label = plot_str[i], color = palette[i])
    
p.legend.location = 'center_right'
p.legend.click_policy = 'hide'
bokeh.io.show(p)


**don't know problem with dna assembly process** ADD RNA POLYMERASES and mrna degradation (take rate params from default_parameters.txt)

## Combine ATP synthase tx, tl, membrane integration, atp synthesis models & ATP USE 
---

In [29]:
ss4_atp_use = createSubsystem('atp_use.xml', subsystemName = 'atp_use')

In [30]:
atp_subsystem_w_use = ss1_atpsyn_txtl + ss2_atpsyn_mi + ss3_atpsynthesis + ss4_atp_use

In [31]:
# SIMULATE WITH ATP SYNTHASE
# Initial conditions
spec_list_10 = ['dna_atpsyn', 'proton_outside_H', 'metabolite_atp']
for i,spec in enumerate(spec_list_10):
    atp_subsystem_w_use.setSpeciesAmount(spec,10)


# timepoints and get results
timepoints = np.linspace(0,10,100)
results, _ = atp_subsystem_w_use.simulateWithBioscrape(timepoints, sbml_warnings = False)

# Get species IDs
ATP_id = atp_subsystem_w_use.getSpeciesByName('metabolite_atp').getId()
ADP_id = atp_subsystem_w_use.getSpeciesByName('metabolite_adp').getId()

outside_H_id = atp_subsystem_w_use.getSpeciesByName('proton_outside_H').getId()
inside_H_id = atp_subsystem_w_use.getSpeciesByName('proton_inside_H').getId()
atp_synthase_b_protein_id = atp_subsystem_w_use.getSpeciesByName('protein_atpsyn_b').getId()
atp_synthase_f_protein_id = atp_subsystem_w_use.getSpeciesByName('protein_atpsyn').getId()

In [32]:
# SIMUALTE WITHOUT ATP SYNTHASE
#ss4_atp_use.setSpeciesAmount('proton_outside_H',10)
ss4_atp_use.setSpeciesAmount('metabolite_atp',10)

timepoints = np.linspace(0,10,100)
results2, _ = ss4_atp_use.simulateWithBioscrape(timepoints, sbml_warnings = False)

ATP_id2 = ss4_atp_use.getSpeciesByName('metabolite_atp').getId()

In [33]:
# PLOT WITH ATP SYNTHASE
p_w_atp = bokeh.plotting.figure(width = 450, height = 350, title = 'with ATP Synthase, ATP Use',
                          x_axis_label = 'time',
                          y_axis_label = 'concentration')
plot_list = [ATP_id, outside_H_id, inside_H_id, atp_synthase_b_protein_id, atp_synthase_f_protein_id]
plot_str = ['atp', 'outside_H', 'inside_H', 'bound atp synthase', 'free atp synthase']
# plot_list = [atp_synthase_b_protein_id, atp_synthase_f_protein_id]
# plot_str = ['atp synthase bound', 'atp synthase free']
for i,item in enumerate(plot_list):
    p_w_atp.line(timepoints, results[item], line_width = 2, legend_label = plot_str[i], color = palette[i])
    

# PLOT WITHOUT ATP SYNTHASE
p_no_atp = bokeh.plotting.figure(width = 450, height = 350, title = 'with ATP Use',
                          x_axis_label = 'time',
                          y_axis_label = 'concentration')
p_no_atp.line(timepoints, results2[ATP_id2], line_width = 2, legend_label = 'atp', color = palette[0])


p_w_atp.legend.location = 'bottom_right'
    
bokeh.io.show(row(p_w_atp, p_no_atp))



Nice we see that there's just a lower steady state when we add atp use, as expected

But how come it's staying regenerated for ever. Once there's no more outside H, no flow in of protons so no ATP synthesis via atp synthase. But atp should still be used by the atp_use subsbml so why isn't atp degrading?

We need to get it to say, once there is 1 atp synthase, something happens.

For the 'SBML for ATP synthesis via ATP synthase membrane protein' we need to include ATP synthase!!

## Assemble SUbSBmls for Proton Pump
---

In [17]:
#ss5_pp_txtl = createSubsystem('proton_pump_tx_tl.xml', subsystemName = 'pp_txtl')
ss5_pp_txtl = createSubsystem('proton_pump_simple_txtl.xml', subsystemName = 'pp_txtl')
ss6_pp_mi = createSubsystem('proton_pump_membrane_integration.xml', subsystemName = 'pp_mi')
ss7_protontransport = createSubsystem('proton_transport_through_proton_pump.xml', subsystemName = 'protontransport')

In [18]:
pp_subsystem = ss5_pp_txtl + ss6_pp_mi + ss7_protontransport

In [19]:
# Initial conditions
spec_list_10 = ['dna_proton_pump', 'proton_outside_H', 'metabolite_atp']
for i,spec in enumerate(spec_list_10):
    pp_subsystem.setSpeciesAmount(spec,10)


# timepoints and get results
timepoints = np.linspace(0,100,100)
results, _ = pp_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

# Get species IDs
ATP_id = pp_subsystem.getSpeciesByName('metabolite_atp').getId()
ADP_id = pp_subsystem.getSpeciesByName('metabolite_adp').getId()

outside_H_id = pp_subsystem.getSpeciesByName('proton_outside_H').getId()
inside_H_id = pp_subsystem.getSpeciesByName('proton_inside_H').getId()
proton_pump_b_protein_id = pp_subsystem.getSpeciesByName('protein_proton_pump_b').getId()
proton_pump_f_protein_id = pp_subsystem.getSpeciesByName('protein_proton_pump').getId()

In [20]:
results[proton_pump_b_protein_id]

0      0.000000
1      0.821687
2      3.287915
3      5.835647
4      7.677573
        ...    
95    10.000000
96    10.000000
97    10.000000
98    10.000000
99    10.000000
Name: protein_proton_pump_b_pp_mibiocrnpyler_865448, Length: 100, dtype: float64

In [21]:
# timepoints = np.linspace(0,10,100)
# results2, _ = pp_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

In [22]:
p_pp = bokeh.plotting.figure(width = 450, height = 350)
plot_list = [ATP_id, outside_H_id, inside_H_id, proton_pump_b_protein_id]
plot_str = ['atp', 'outside_H', 'inside_H', 'bound proton pump']
for i,item in enumerate(plot_list):
    p_pp.line(timepoints, results[item], line_width = 2, legend_label = plot_str[i], color = palette[i])
    
p_pp.legend.click_policy = 'hide'
bokeh.io.show(p_pp)

## Combine ATP Synthase, ATP Use, and Proton pump
---

In [23]:
entire_subsystem = ss1_atpsyn_txtl + ss2_atpsyn_mi + ss3_atpsynthesis + ss4_atp_use + ss5_pp_txtl + ss6_pp_mi + ss7_protontransport

In [24]:
# Initial conditions
spec_list_10 = ['dna_proton_pump', 'dna_atpsyn','proton_outside_H', 'metabolite_atp']
for i,spec in enumerate(spec_list_10):
    entire_subsystem.setSpeciesAmount(spec,10)


# timepoints and get results
timepoints = np.linspace(0,10,100)
results, _ = entire_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

# Get species IDs
ATP_id = entire_subsystem.getSpeciesByName('metabolite_atp').getId()
ADP_id = entire_subsystem.getSpeciesByName('metabolite_adp').getId()

outside_H_id = entire_subsystem.getSpeciesByName('proton_outside_H').getId()
inside_H_id = entire_subsystem.getSpeciesByName('proton_inside_H').getId()
proton_pump_b_protein_id = entire_subsystem.getSpeciesByName('protein_proton_pump_b').getId()
proton_pump_f_protein_id = entire_subsystem.getSpeciesByName('protein_proton_pump').getId()

In [25]:
timepoints = np.linspace(0,10,100)
results3, _ = entire_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

In [26]:
p3 = bokeh.plotting.figure(width = 450, height = 350, title = 'with ATP Synthase, Proton Pump, ATP Use',
                          x_axis_label = 'time',
                          y_axis_label = 'concentration')
plot_list = [ATP_id, outside_H_id, inside_H_id, proton_pump_b_protein_id]
plot_str = ['atp', 'outside_H', 'inside_H', 'bound proton pump']
for i,item in enumerate(plot_list):
    p3.line(timepoints, results3[item], line_width = 2, legend_label = plot_str[i], color = palette[i])
    
p3.legend.click_policy = 'hide'
p3.legend.location = 'bottom_right'
bokeh.io.show(p3)

## Compare with atpsyn+protonpump, only atpsyn, only atpuse
---

In [27]:
bokeh.io.show(row(p3,p_w_atp, p_no_atp))

Why does plot 2 have higher steady state, how does temperature affect?

In [35]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab,scipy,biocrnpyler,bioscrape,colorcet,subsbml
  

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
bokeh 1.4.0
jupyterlab 1.2.6
scipy 1.4.1
biocrnpyler 0.2.1
bioscrape 1.0.0
colorcet 2.0.2
subsbml unknown
